In [1]:
import pickle
import numpy as np
from scipy.optimize import linprog
from numpy.linalg import solve      # from scipy.linalg import solve

In [2]:
# load exercize into memory
EXAMPLE_NUM = 2

with open(f"Data/problem{EXAMPLE_NUM}.pickle", "rb") as f:
    E_sparse, Q_diag, q, b, u, solution = pickle.load(f)
Q = np.diag(Q_diag)
E = E_sparse.toarray()
u = np.nan_to_num(u, posinf=1e6)

In [3]:
n, m = E.shape
n, m

(20, 108)

In [4]:
# set absolute tollerance for most checks
ATOL = 1e-6

### Problem Formulation
$\min_x \ f(x) = \frac{1}{2}x^T Q x + q^T x$

constrained by: $\quad E x = b, \quad -x \leq 0, \quad x \leq u$

### Additionally we know:
- equality constraint functions: $\quad h(x) = e^T x - b$
- inquality constraint functions: $\quad g_L(x) = -x, \quad g_U(x) = x - u$
- cost function gradient: $\quad \nabla{f(x)} = Q x + q$
- constraint function gradients: $\quad \nabla{h(x)}=E, \quad \nabla{g_L(x)}= -I_m, \quad \nabla{g_U(x)}= I_m$
- x is a solution if KKT holds:
    - stationarity: $\quad \nabla{f(x)} + \lambda^T \nabla{h(x)} + \mu^T \nabla{g(x)} = 0$
    - primal/dual feasibility + complementary slackness: $\quad g(x) \leq 0, \quad h(x) = 0, \quad \mu \geq 0, \quad \mu^T g(x) = 0$

### Active Set formulation
- set of lower bounded indices: $\quad A_L = \{ i \mid x_i = 0 \}$
- set of upper bounded indices: $\quad A_U = \{ i \mid x_i = u_i \}$
- set of free indices: $\quad F = I \setminus (A_L \cup A_U), \quad I = \{ 1,2,...,n \}$

### Utility Functions

In [5]:
def assert_contraints(x, tol=1e-6):
    assert np.allclose(E.dot(x), b, atol=tol), "equality is broken"
    assert np.allclose(-tol < x, True),        "lower bound is broken"
    assert np.allclose(x < u + tol, True),     "upper bound is broken"
    return True

In [6]:
def get_cost(x):
    return 0.5*x.dot(Q).dot(x) + x.dot(q)

In [7]:
def get_active_box_constraints(x, lower=0,  upper=1e6, tol=1e-6):
    lower_bounded = np.argwhere(np.abs(x-lower) < tol).flatten()
    upper_bounded = np.argwhere(np.abs(x-upper) < tol).flatten()
    return lower_bounded, upper_bounded

In [8]:
def print_solution_and_cost(x, fx, i=None):
    print(f"x^({i}) = {x}")
    print(f"f(x^({i})) = {fx}")

In [9]:
def print_sets(AL=None, AU=None, A=None, F=None, i=None):
    if AL is not None:
        print(f"A_L^({i}) = {AL}")
    if AU is not None:
        print(f"A_U^({i}) = {AU}")
    if A is not None:
        print(f"A^({i}) = {A}")
    if F is not None:
        print(f"F^({i}) = {F}")

In [10]:
def get_matrix_encoding(E):
    """Matrix encoding used when computing the Connected Components
    @returns nn_map, e_map
    nn_map : dict (key=nodeID, value=adjacent_nodeIDs)
    e_map  : dict (key=edgeID, value=(in_nodeID, out_nodeID))
    """
    # node-node connections = {ROWS | for each node: outgoing and incoming nodes}
    nn_map = {}
    for i, row in enumerate(E):
        outgoing = np.argwhere(row < -0.9).flatten(), 
        incoming = np.argwhere(row >  0.9).flatten(),
        nn_map[i] = np.union1d(outgoing, incoming)
    # edges = {COLS | for each edge: in node, out node}
    e_map = {}
    for i, col in enumerate(E.T):
        e_map[i] = (np.argwhere(col < -0.9).flatten()[0], np.argwhere(col > 0.9).flatten()[0])
    return nn_map, e_map

In [11]:
def get_subgraph_connected_components(active_edges):
    """Compute connected components of the subgraph given by keeping all the
    original nodes and edges only the ones in 'active_edges'.
    @returns ccs : list of connected components
    
    Some values are implictly passed: [n, m, nn_map, e_map]
    n : # nodes, m : # edges, 
    nn_map, e_map are the ones defined in get_matrix_encoding.
    """
    # 

    # initial values
    visited = np.full(n, False)
    ccs = []
    
    # depth first search (subgraph given by active_edges)
    def dfs(nid, cc):
        for eid in nn_map[nid]:
            if eid in active_edges:
                in_nid, out_nid = e_map[eid]
                c_nid = out_nid if out_nid != nid else in_nid
                if not visited[c_nid]:
                    visited[c_nid] = True
                    cc.append(c_nid)
                    dfs(out_nid, cc)
    # start dfs for each node
    for nid in range(n):
        if not visited[nid]:
            visited[nid] = True
            new_cc = [nid]
            dfs(nid, new_cc)
            ccs.append(new_cc)
    return ccs

In [12]:
def quadsolve(Q_diag, q, A_eq, b_eq=0, tol=1e-6):
    """Computes the solution of an equality constrained quadratic program given as:
        min_x(1/2*xQx + qx) constrained by A_eq x = b_eq.
    
    @param Q_diag : the diagonal of Q, positive definite
    @param q : (m,) array
    @param A_eq : (n x m) matrix with n linearly indipendent rows 
    @param b_eq : (n,) array [default is 0]
    
    @returns x, lambdas
    x : solution of the quadratic program
    lambdas : lagrangian multipliers
    """
    Q_diag_inv = 1 / Q_diag
    AQ_inv = np.multiply(A_eq, Q_diag_inv)
    _b = -b_eq - AQ_inv.dot(q)
    _A = AQ_inv.dot(A_eq.T)
    
    lambdas = solve(_A, _b)
    x = -np.multiply(Q_diag_inv, q + A_eq.T.dot(lambdas))
    
    return x, lambdas

In [13]:
def assert_step(p, an, ae, tol=1e-6):
    """Assert the step is valid.
    @param p : step
    @param an: linearly independent contraint indices
    @param ae: free variable set
    """
    assert np.allclose(E[np.ix_(an, ae)].dot(p), 0, atol=tol), "equality is broken"

In [14]:
def get_mus(x, AL, AU):
    # mu = -g(x) (grad_fx)
    grad_fx = Q.dot(x) + q
    return grad_fx[AL], -grad_fx[AU]

In [15]:
def get_linearly_independent_constraints(ccs):
    """Returns a set of linearly independent constraints.
    @param ccs : Connected Components of the subgraph
    @returns lic : linearly independent contraint indices
    """
    lic = []
    for cc in ccs:
        lic.extend(cc[:-1])
    return np.array(lic)

In [16]:
def get_lagrangian_multipliers(x, an, ae, lb, ub):
    """Given a square singular matrix of all active contraints
    returns associated lagrangian multipliers.
    @param x : solution
    @param an: linearly independent contraint indices
    @param ae: free variable set
    @param lb: lower bounded indices
    @param ub: upper bounded indices
    @returns lambdas, muL, muU
    lambdas : lagrangian multipliers for equalities
    muL : lagrangian multipliers for lower bounded indices
    muU : lagrangian multipliers for upper bounded indices
    """
    # get sizes to split solution
    sizes = [0, an.size, lb.size, ub.size]
    splits = np.array(sizes).cumsum()
    
    assert ae.size == an.size, "matrix is not square"
    
    # solve lagrangian multipliers
    sol = solve(np.block([E[an].T, -I[lb].T, I[ub].T]), -Q.dot(x)-q)
    lambdas = sol[splits[0]: splits[1]]
    muL     = sol[splits[1]: splits[2]]
    muU     = sol[splits[2]: splits[3]]
    
    return lambdas, muL, muU

In [17]:
def get_lagrangian_multipliers_full(x, an, ae, lb, ub):
    """Given a linearly independent matrix of all active contraints
    returns associated lagrangian multipliers.
    @param x : solution
    @param an: linearly independent contraint indices
    @param ae: free variable set
    @param lb: lower bounded indices
    @param ub: upper bounded indices
    @returns lambdas, muL, muU
    lambdas : lagrangian multipliers for equalities
    muL : lagrangian multipliers for lower bounded indices
    muU : lagrangian multipliers for upper bounded indices
    """
    I = np.eye(m,m)
    O = np.zeros((m,m))
    _A = np.block([
        [Q  , E.T      , -I       , I        ],
        [E  , O[:n,:n] , O[:n,:m] , O[:n,:m] ],
        [-I , O[:m,:n] , O[:m,:m] , O[:m,:m] ],
        [I  , O[:m,:n] , O[:m,:m] , O[:m,:m] ]
    ])
    _b = np.concatenate([-q, b, np.zeros(m), u])
    del O

    sizes = [0, m, an.size, AL_k.size, AU_k.size]
    splits = np.array(sizes).cumsum()
    idx = np.concatenate([np.arange(m), m + an, (m+n) + AL_k, (2*m + n) + AU_k])
    sol = solve(_A[np.ix_(idx, idx)], _b[idx])
    x       = sol[splits[0]: splits[1]]
    lambdas = sol[splits[1]: splits[2]]
    muL     = sol[splits[2]: splits[3]]
    muU     = sol[splits[3]: splits[4]]
    del sizes, splits
    assert np.allclose(x, x_k, atol=ATOL), "x != x_k"
    return lambdas, muL, muU

### 1. Compute feasible starting point $x^{(0)}$ and active sets $A_L = \{ i \mid x_i = 0 \}$, $A_U = \{ i \mid x_i = u_i \}$

In [18]:
## implement symplex method instead of using
res = linprog(np.ones(m), A_eq=E[:-1], b_eq=b[:-1], bounds=[(0, x) for x in u], ) #method="simplex")

# get inital x and check constraints (primal feasability)
assert res["success"], "[ERROR] simplex method failed"
x_k = res["x"]
assert_contraints(x_k, tol=ATOL)

# print current solution and cost
fx_k = get_cost(x_k)
print_solution_and_cost(x_k, fx_k, i=0)

# get current active sets 
AL_k, AU_k = get_active_box_constraints(x_k, 0, u, tol=ATOL)
A_k = np.union1d(AL_k, AU_k)
F_k = np.setdiff1d(np.arange(m), A_k)

# print active sets
print_sets(AL_k, AU_k, A_k, F_k, i=0)

x^(0) = [6.41347916e-10 3.33545451e+00 3.01638927e-10 8.24251597e-10
 4.26766146e-10 7.58997463e-10 3.28299338e-10 1.66454549e+00
 2.29758260e-10 3.33680577e-09 7.83642927e-10 8.78667308e-10
 6.27762682e-10 7.10516999e-10 7.78697941e-10 1.07082156e-09
 7.40220715e-10 2.08291951e-09 8.83884678e-10 1.58107669e+00
 1.11180616e+00 2.65931105e-09 1.52119431e-09 4.66815901e-09
 4.35645833e-09 6.42571648e-01 1.73431584e-09 1.66454549e+00
 3.29355880e-10 4.07286580e-10 3.73282982e-10 3.96790516e-10
 2.62445485e-10 1.19890697e-09 7.76533426e-10 1.15006980e-09
 1.03958405e-09 6.18024631e-10 3.73403054e-09 1.49167777e-09
 9.29058603e-10 8.10305781e-10 1.07847746e-09 1.04793117e-09
 4.52862914e-10 9.92569537e-10 8.86722733e-10 6.94474307e-10
 8.61070635e-10 2.30711715e+00 5.94271365e-10 5.53809930e-10
 5.85456107e-10 7.93619784e-10 5.25000116e-10 5.07564603e-10
 4.64494314e-10 1.17370571e-09 1.16850971e-09 5.79348233e-10
 6.46087762e-10 7.87827248e-10 6.81869577e-10 8.07707802e-10
 9.43387895e-10 

In [19]:
# get matrix encoding
nn_map, e_map = get_matrix_encoding(E)

In [20]:
# compute a single indentity matrix for indexing
I = np.eye(m)

## 2. Active Set Method
### 2.1. Compute maximal step $p^{(k)}$
### 2.2a if ($p^{(k)} \approx 0$) then Compute lagrangian multipliers $\mu^{(k)}$
#### 2.2a.1a if ($\mu^{(k)} \approx 0$) then STOP
#### 2.2a.1b else remove indices with most negative $\mu^{(k)}$ from $A_L^{(k)}, A_U^{(k)}$
### 2.2b else Compute maximum step size $\alpha^{(k)}$ such that $x^{(k+1)}$ is feasible and update $A_L^{(k)}, A_U^{(k)}$

In [21]:
def primal_algorithm(i):
    global x_k, AL_k, AU_k, A_k, F_k
    
    # get linerarly independent contraints
    ccs = get_subgraph_connected_components(F_k)
    lic = get_linearly_independent_constraints(ccs)
    
    # 2.1 : compute step p^k
    p_k, _ = quadsolve(
        Q_diag[F_k],                       # Q_FF
        Q_diag[F_k] * x_k[F_k] + q[F_k],   # g^k_F
        E[np.ix_(lic, F_k)],               # E_GF
        tol=ATOL
    )
    assert_step(p_k, lic, F_k)
    print(f"p^({i}) = {p_k}")
    
    # 2.2a : check if p^k == 0
    if np.allclose(p_k, 0, atol=ATOL):
        assert lic.size == F_k.size, "Issues"
        
        _, muL, muU = get_lagrangian_multipliers(x_k, lic, F_k, AL_k, AU_k)
        
        # 2.2a.1a : check if mu^k > 0
        min_muL = np.min(muL, initial=np.inf)
        min_muU = np.min(muU, initial=np.inf)
        if min_muL > -ATOL and min_muU > -ATOL:
            return True
        
        print(f"mu_L^({i}) = {muL}")
        print(f"mu_U^({i}) = {muU}")
        
        min_mu = np.min([min_muL, min_muU])
        
        jsL = np.argwhere(np.abs(muL - min_mu) < ATOL).flatten()
        jsU = np.argwhere(np.abs(muU - min_mu) < ATOL).flatten()
        
        # 2.2a.1a : remove most negative indices
        AL_k = np.setdiff1d(AL_k, AL_k[jsL])
        AU_k = np.setdiff1d(AU_k, AU_k[jsU])
        A_k  = np.union1d(AL_k, AU_k)
        F_k  = np.setdiff1d(np.arange(m), A_k)

    # 2.2b : compute solution update
    else:
        p_k_nonzero = np.ma.array(p_k, mask=np.logical_and(p_k < 1e-6, p_k > -1e-6))
        # assert p_k_nonzero.mask.sum() > 0, "Issues 2"
        
        # max step for constraint (ax <= c) if (ap > 0) is given by: (c - ax) / ap
        alphasL = np.ma.array(
            (u[F_k] - x_k[F_k]) / p_k,  
            mask=~np.logical_and(~p_k_nonzero.mask, p_k > 0)
        ) # a= 1, c=u
        alphasU = np.ma.array(
            -x_k[F_k] / p_k,  
            mask=~np.logical_and(~p_k_nonzero.mask, p_k < 0)
        ) # a=-1, c=0
        
        min_alphasL = alphasL.min(fill_value=np.inf)
        min_alphasU = alphasU.min(fill_value=np.inf)
        
        alpha = np.min([1, min_alphasL, min_alphasU])
        assert alpha > ATOL, "Issues 3"
        
        # update solution
        x_k[F_k] = x_k[F_k] + alpha*p_k
        
        # update active sets
        AL_k, AU_k = get_active_box_constraints(x_k, 0, u, tol=ATOL)
        A_k = np.union1d(AL_k, AU_k)
        F_k = np.setdiff1d(np.arange(m), A_k)
    return False

In [22]:
ITER_LIM = 100
for i in range(ITER_LIM):
    print(f"--------- ITERATION {i} ---------")
    print_solution_and_cost(x_k, get_cost(x_k), i)
    print(f"distance from solution: {np.linalg.norm(x_k - solution)}")
    print_sets(AL_k, AU_k, A_k, F_k, i)
    print(f"#AL^{i} = {AL_k.size}, #AU^{i} = {AU_k.size}, #A^{i} = {A_k.size}, #F^{i} = {F_k.size}")
    if primal_algorithm(i):
        print("solution found!")
        break

--------- ITERATION 0 ---------
x^(0) = [6.41347916e-10 3.33545451e+00 3.01638927e-10 8.24251597e-10
 4.26766146e-10 7.58997463e-10 3.28299338e-10 1.66454549e+00
 2.29758260e-10 3.33680577e-09 7.83642927e-10 8.78667308e-10
 6.27762682e-10 7.10516999e-10 7.78697941e-10 1.07082156e-09
 7.40220715e-10 2.08291951e-09 8.83884678e-10 1.58107669e+00
 1.11180616e+00 2.65931105e-09 1.52119431e-09 4.66815901e-09
 4.35645833e-09 6.42571648e-01 1.73431584e-09 1.66454549e+00
 3.29355880e-10 4.07286580e-10 3.73282982e-10 3.96790516e-10
 2.62445485e-10 1.19890697e-09 7.76533426e-10 1.15006980e-09
 1.03958405e-09 6.18024631e-10 3.73403054e-09 1.49167777e-09
 9.29058603e-10 8.10305781e-10 1.07847746e-09 1.04793117e-09
 4.52862914e-10 9.92569537e-10 8.86722733e-10 6.94474307e-10
 8.61070635e-10 2.30711715e+00 5.94271365e-10 5.53809930e-10
 5.85456107e-10 7.93619784e-10 5.25000116e-10 5.07564603e-10
 4.64494314e-10 1.17370571e-09 1.16850971e-09 5.79348233e-10
 6.46087762e-10 7.87827248e-10 6.81869577e-10

/tmp/ipykernel_447/3601527575.py:51: RuntimeWarning: divide by zero encountered in true_divide
  (u[F_k] - x_k[F_k]) / p_k,
/tmp/ipykernel_447/3601527575.py:55: RuntimeWarning: divide by zero encountered in true_divide
  -x_k[F_k] / p_k,
/home/david/mambaforge/envs/cm/lib/python3.10/site-packages/numpy/core/fromnumeric.py:86: UserWarning: Warning: converting a masked element to nan.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


AssertionError: Issues 3

In [ ]:
x_k

In [ ]:
u